In [ ]:
#import SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import desc

In [ ]:
#create spar session object
spark=SparkSession.builder.appName('data_processing').getOrCreate()

In [ ]:
# Load csv Dataset 
df=spark.read.csv('s3://datasetsfperezm1/covid/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)

In [ ]:
#2.1 columns of dataframe
df.columns

In [ ]:
#2.2 printSchema - types 
df.printSchema()

In [ ]:
#2.3 select some columns
df.select('fecha reporte web','Nombre departamento','Edad').show(5)

In [ ]:
#2.4 Add new Column
df = df.withColumn("EdadNegativa", -1*F.col("Edad"))
df.select('EdadNegativa','Edad').show(5)

In [ ]:
#2.5 Delete column
df2 = df.drop('EdadNegativa')
df2.printSchema()

In [ ]:
#2.6 filter data
df.filter(df.Edad == 20).show(5, truncate=False)

In [ ]:
#2.7 Lambda function
rdd2 = df.rdd.map(lambda x: (x["Edad"]+","+x["Unidad de medida de edad"]))

def func1(x):
    edadCompleta=x["Edad"]+","+x["Unidad de medida de edad"]
    return (edadCompleta)

rdd2=df.rdd.map(lambda x: func1(x))
print(rdd2)

In [ ]:
#3

oldColumns = data.schema.names
newColumns = ["FechaReporteWeb", "IdCaso", "FechaNotificación", "CodDIVIPOLAdepartamento", "Departamento", 
              "CodigoDIVIPOLAmunicipio", "Municipio", "Edad"
 |-- Unidad de medida de edad
 |-- Sexo
 |-- Tipo de contagio
 |-- Ubicación del caso
 |-- Estado
 |-- Código ISO del país
 |-- Nombre del país
 |-- Recuperado
 |-- Fecha de inicio de síntomas
 |-- Fecha de muerte
 |-- Fecha de diagnóstico
 |-- Fecha de recuperación
 |-- Tipo de recuperación
 |-- Pertenencia étnica
 |-- Nombre del grupo étnico

df = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), xrange(len(oldColumns)), data)
df.printSchema()
df.show()


In [ ]:
#3.1 - 10 departamentos con más casos de covid en Colombia ordenados de mayor a menor.
tres_uno = df.groupBy("Nombre departamento").count().orderBy('count', ascending=False)
tres_uno.coalesce(1).write.csv("s3a://datasetsfperezm1/tres/tres_uno", header = True)

In [ ]:
#3.2 - 10 días con más casos de covid en Colombia ordenados de mayor a menor.
tres_dos = df.groupBy("Nombre municipio").count().orderBy('count', ascending=False)
tres_dos.coalesce(1).write.csv("s3a://datasetsfperezm1/tres/tres_dos", header = True)

In [ ]:
#3.3 - 10 días con más casos de covid en Colombia ordenados de mayor a menor.
tres_tres = df.groupBy("fecha reporte web").count().orderBy('count', ascending=False)
tres_tres.coalesce(1).write.csv("s3a://datasetsfperezm1/tres/tres_tres", header = True)

In [ ]:
#3.4 - Distribución de casos por edades de covid en Colombia.
tot = df.count()
tres_cuatro = df.groupBy("Edad") \
              .count() \
              .withColumnRenamed('count', 'cnt_per_group') \
              .withColumn('perc_of_count_total', (F.col('cnt_per_group') / tot) * 100 ) \
              .orderBy('count', ascending=False)
tres_cuatro.coalesce(1).write.csv("s3a://datasetsfperezm1/tres/tres_cuatro", header = True)

In [ ]:
#3.5 - Numero de casos entre hombres y mujeres
tres_cinco = df.groupBy("Sexo").count().orderBy('count', ascending=False)
tres_cinco.coalesce(1).write.csv("s3a://datasetsfperezm1/tres/tres_cinco", header = True)